In [1]:
import findspark
findspark.init()


In [2]:
full_text=sc.textFile('shakespeare.txt')
full_text.cache()

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [243]:
import re
import random
parsed_text=(full_text
             .map(lambda x: re.sub(r"\d+", "", x))
             .map(lambda x: re.sub(r"[A-Z]+\.", "", x))
             .map(lambda x: re.sub(r"[A-Z]+$|[A-Z]+[^a-zA-Z0-9_\']", "", x)))

In [253]:
words=(parsed_text.flatMap(lambda x:re.split(' ',x)).map(lambda x:x.strip(".,:;?-][!()'"))
       .filter(lambda x: x != '')
       .zipWithIndex()
       .filter(lambda x:x[1]>773)
       .map(lambda x: x[0]))
words_array=words.collect()
indices=sc.range(words.count()-2)
triples=indices.map(lambda x:((words_array[x],words_array[x+1]),words_array[x+2]))
markov_words=triples.groupByKey().mapValues(lambda words:[(word,list(words).count(word)) for word in set(words)])

[[(u'take', 1)]]

In [261]:
starting_words = markov_words.takeSample(False,10)
sentences=[]
for starter in starting_words:
    sentence=list(starter[0])
    for i in range(2,20):
        print i
        possible_words=markov_words.map(lambda x:x).lookup((sentence[i-2],sentence[i-1]))[0]
        if len(possible_words)==1:
            sentence.append(possible_words[0][0])
        else:
            total=sum(count for word,count in possible_words)
            rand=random.uniform(0,total)
            threshold = 0
            for word, count in possible_words:
                if threshold + count > rand:
                    sentence.append(word)
                    break
                threshold += count
    sentences.append(sentence)
        

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [262]:
sentences

[[u'were',
  u'lame',
  u'and',
  u'impotent',
  u'conclusion',
  u'Do',
  u'not',
  u'you',
  u'go',
  u'in',
  u'clouted',
  u'shoon',
  u'For',
  u'they',
  u'have',
  u'their',
  u'honest',
  u'wills',
  u'Which',
  u'seasons'],
 [u'Dark-working',
  u'sorcerers',
  u'that',
  u'change',
  u'is',
  u'from',
  u'me',
  u'what',
  u'they',
  u'hate',
  u'us',
  u'youth',
  u'Down',
  u'with',
  u'that',
  u'base',
  u'wretch',
  u'One',
  u'bred',
  u'of'],
 [u'You',
  u'three',
  u'Berowne',
  u'Dumain',
  u'and',
  u'Longaville',
  u'Have',
  u'sworn',
  u'for',
  u'three',
  u'months',
  u'you',
  u'told',
  u'me',
  u'they',
  u'live',
  u'like',
  u'venom',
  u'where',
  u'no'],
 [u'that',
  u'Prince',
  u'Harry',
  u'in',
  u'the',
  u'grapple',
  u'boarded',
  u'them',
  u'On',
  u'the',
  u'white',
  u'wheat',
  u'and',
  u'hurts',
  u'the',
  u'poor',
  u'souls',
  u'for',
  u'this',
  u'siege'],
 [u'spit',
  u'upon',
  u'my',
  u'sword',
  u'John',
  u'But',
  u'soft',
  u'W